In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
import tensorflow as tf
from tensorflow import keras

In [2]:
df = pd.read_pickle("../project_data/every_ten_seconds.pkl")
df.reset_index(inplace = True, drop = True)
df['time'] = df['time'].astype(float)
# we need to order by time to make splitting easier
df.sort_values(by = ['time'], ignore_index = True, inplace = True)

We use the cutoff point of 0.08 to distinguish between intoxicated and sober classes. It is the legal limit for intoxication while driving and according to the National Institute on Alcohol Abuse and Alcoholism it marks the onset of a binge drinking event.

In [3]:
# prepare target
df['TAC_Reading_binary'] = np.array([1 if tac >= 0.08 else 0 for tac in df['TAC_Reading'].values])

We use 3-fold cross-validation to evaluate each model's performance. We implement this manually (rather on relying on scikit-learn's implementations), because our data is not i.i.d. Scikit-learn implementations usually shuffle the data or make it difficult to guarantee non-overlapping folds. In our case, shuffling the data leads to an inflated validation score, as the model will be tested on samples that are close in time (and hence artificially similar) to its training samples.  

In [5]:
# store indices for each split
train_1_indices = np.arange(8712, len(df))
test_1_indices = np.arange(0, 8712)
train_2_indices = np.concatenate((np.arange(0, 8712), np.arange(8712*2, len(df))))
test_2_indices = np.arange(8712, 8712*2)
train_3_indices = np.arange(0, 8712*2)
test_3_indices = np.arange(8712*2, len(df))

# how many training observations in each fold?
print('there are', len(df), 'observations in the training data')
print('train_fold_1 will have', len(train_1_indices), 'observations')
print('train_fold_2 will have', len(train_2_indices), 'observations')
print('train_fold_3 will have', len(train_3_indices), 'observations')
print('test_fold_1 will have', len(test_1_indices), 'observations')
print('test_fold_2 will have', len(test_2_indices), 'observations')
print('test_fold_3 will have', len(test_3_indices), 'observations')

there are 26137 observations in the training data
train_fold_1 will have 17425 observations
train_fold_2 will have 17425 observations
train_fold_3 will have 17424 observations
test_fold_1 will have 8712 observations
test_fold_2 will have 8712 observations
test_fold_3 will have 8713 observations


In [67]:
def standardize_inputs(X_train, X_test):
    '''
    Standardize a train-test split using mean and standard deviation estimates from training data.
    Returns a tuple of standardized training and test matrices.
    '''
    scale = StandardScaler()
    scale.fit(X_train)
    X_train_scaled = scale.transform(X_train)
    X_test_scaled = scale.transform(X_test)
    return(X_train_scaled, X_test_scaled)

def data_for_modeling(train_indices, test_indices, standardize = True, multi_input = False, multi_output = False, data = df):
    '''
    Get the training and test arrays for predictors and target to feed into your model.
    Objects returned in a tuple in the following order: (X_train_objects, ..., X_test_objects, ..., y_train_objects, ..., y_test_objects)
    Standardizes X appropriately, unless standardize = False, in which case X is not standardized.
    If multi_input = True, multiple X objects are returned (to be used in wide and deep network).
    If multi_output = True, multiple y objects are returned (to be used in multi-task network).
    Does not support both multi_input and multi_output.
    '''
    # prepare target
    y_train = df.iloc[train_indices]['TAC_Reading_binary'].values
    y_test = df.iloc[test_indices]['TAC_Reading_binary'].values
    if multi_output:
        y_train_classification = df.iloc[train_indices]['TAC_Reading_binary'].values
        y_train_regression = df.iloc[train_indices]['TAC_Reading'].values
        y_test_classification = df.iloc[test_indices]['TAC_Reading_binary'].values
        y_test_regression = df.iloc[test_indices]['TAC_Reading'].values
    
    # prepare predictors
    X_train = df.iloc[train_indices].drop(['time', 'pid', 'TAC_Reading', 'TAC_Reading_binary'], axis = 1)
    X_test = df.iloc[test_indices].drop(['time', 'pid', 'TAC_Reading', 'TAC_Reading_binary'], axis = 1)
    if multi_input:
        X_train_wide = X_train.drop(['x', 'y', 'z'], axis = 1)
        X_test_wide = X_test.drop(['x', 'y', 'z'], axis = 1)    
        X_train_deep = X_train.loc[:, ['x', 'y', 'z']]
        X_test_deep = X_test.loc[:, ['x', 'y', 'z']]
    
    # standardization
    if standardize:
        if multi_input:
            X_train_wide_scaled, X_test_wide_scaled = standardize_inputs(X_train_wide, X_test_wide)
            X_train_deep_scaled, X_test_deep_scaled = standardize_inputs(X_train_deep, X_test_deep)
        else:
            X_train_scaled, X_test_scaled = standardize_inputs(X_train, X_test)

    # returns
    if multi_input:
        if standardize:
            return(X_train_wide_scaled, X_train_deep_scaled, X_test_wide_scaled, X_test_deep_scaled, y_train, y_test)
        else:
            return(X_train_wide, X_train_deep, X_test_wide, X_test_deep, y_train, y_test)
    elif multi_output:
        if standardize:
            return(X_train_scaled, X_test_scaled, y_train_classification, y_train_regression, y_test_classification, y_test_regression)
        else:
            return(X_train, X_test, y_train_classification, y_train_regression, y_test_classification, y_test_regression)
    else:
        if standardize:
            return(X_train_scaled, X_test_scaled, y_train, y_test)
        else:
            return(X_train, X_test, y_train, y_test)

# Modeling

In [17]:
# containers to store results
model = []
fold = []
accuracy = []
precision_intox = []
precision_sober = []
recall_intox = []
recall_sober = []
support_sober = []
support_intox = []

results = {'model': model, 'fold': fold, 'accuracy' : accuracy, 'precision (intoxicated)': precision_intox,
          'precision (sober)': precision_sober, 'recall (intoxicated)': recall_intox, 'recall (sober)': recall_sober, 
           'support (sober)': support_sober, 'support (intox)': support_intox}

## Model #1 Regular feedforward NN

In [7]:
def train_regular_feedforward(X_train, X_test, y_train, y_test, name):
    '''
    Creates regular feedforward NN and fits it to supplied data. 
    Returns fitted model.
    Name specifies filename of model saved at checkpoints.
    '''
    # specify network architecture
    model = keras.models.Sequential([
        keras.layers.Input(shape = (99,)),
        keras.layers.Dropout(rate = 0.3),
        keras.layers.Dense(units = 300, activation = "selu", kernel_initializer = "lecun_normal"),
        keras.layers.Dropout(rate = 0.3),
        keras.layers.Dense(units = 300, activation = "selu", kernel_initializer = "lecun_normal"),
        keras.layers.Dropout(rate = 0.3),
        keras.layers.Dense(units = 300, activation = "selu", kernel_initializer = "lecun_normal"),
        keras.layers.Dropout(rate = 0.3),
        keras.layers.Dense(units = 1, activation = "sigmoid")
    ])
    
    # compilation
    optimizer = keras.optimizers.SGD(momentum = 0.9, nesterov = True)
    model.compile(loss = 'binary_crossentropy', optimizer = optimizer, metrics = ['accuracy'])
    
    # callbacks
    # exponential learning rate scheduler
    def exponential_decay(lr0, s):
        def exponential_decay_fn(epoch):
            return lr0 * 0.1**(epoch / s)
        return exponential_decay_fn

    exponential_decay_fn = exponential_decay(lr0=0.01, s=20)
    lr_scheduler = keras.callbacks.LearningRateScheduler(exponential_decay_fn)
    checkpoint_cb = keras.callbacks.ModelCheckpoint('model_checkpoints/' + 'regular_feedforward_' + name + '.h5', save_best_only = True, save_weights_only = False)
    early_stopping_cb = keras.callbacks.EarlyStopping(patience = 15, restore_best_weights = True)
    
    model.fit(X_train, y_train, batch_size = 32, epochs = 50, validation_data = (X_test, y_test),
                        callbacks = [checkpoint_cb, lr_scheduler, early_stopping_cb], verbose = 2)
    
    return(model)

**Fold 1**

In [12]:
# get data
X_train, X_test, y_train, y_test = data_for_modeling(train_1_indices, test_1_indices)
# fit model
feedforward_1 = train_regular_feedforward(X_train, X_test, y_train, y_test, 'fold_1')
# predict labels
pred_prob = feedforward_1.predict(X_test)
pred = np.array([1 if prob >= 0.5 else 0 for prob in pred_prob])

Train on 17425 samples, validate on 8712 samples
Epoch 1/50
17425/17425 - 19s - loss: 0.3436 - accuracy: 0.8636 - val_loss: 1.3535 - val_accuracy: 0.5527
Epoch 2/50
17425/17425 - 20s - loss: 0.2661 - accuracy: 0.8826 - val_loss: 1.3310 - val_accuracy: 0.5733
Epoch 3/50
17425/17425 - 16s - loss: 0.2524 - accuracy: 0.8886 - val_loss: 1.2886 - val_accuracy: 0.5896
Epoch 4/50
17425/17425 - 15s - loss: 0.2358 - accuracy: 0.8898 - val_loss: 1.1185 - val_accuracy: 0.5658
Epoch 5/50
17425/17425 - 15s - loss: 0.2299 - accuracy: 0.8921 - val_loss: 1.3756 - val_accuracy: 0.5853
Epoch 6/50
17425/17425 - 15s - loss: 0.2214 - accuracy: 0.8976 - val_loss: 1.2982 - val_accuracy: 0.5880
Epoch 7/50
17425/17425 - 16s - loss: 0.2154 - accuracy: 0.8971 - val_loss: 1.3047 - val_accuracy: 0.5924
Epoch 8/50
17425/17425 - 17s - loss: 0.2098 - accuracy: 0.9021 - val_loss: 1.2574 - val_accuracy: 0.5900
Epoch 9/50
17425/17425 - 16s - loss: 0.1996 - accuracy: 0.9081 - val_loss: 1.1974 - val_accuracy: 0.5562
Epoch 

In [18]:
# record results
model += ['regular feedforward']
fold += [1]
accuracy += [accuracy_score(y_test, pred)]
precision_intox += [precision_recall_fscore_support(y_test, pred)[0][1]]
precision_sober += [precision_recall_fscore_support(y_test, pred)[0][0]]
recall_intox += [precision_recall_fscore_support(y_test, pred)[1][1]]
recall_sober += [precision_recall_fscore_support(y_test, pred)[1][0]]
support_sober += [precision_recall_fscore_support(y_test, pred)[3][0]]
support_intox += [precision_recall_fscore_support(y_test, pred)[3][1]]

**Fold 2**

In [20]:
# get data
X_train, X_test, y_train, y_test = data_for_modeling(train_2_indices, test_2_indices)
# fit model
feedforward_2 = train_regular_feedforward(X_train, X_test, y_train, y_test, 'fold_2')
# predict labels
pred_prob = feedforward_2.predict(X_test)
pred = np.array([1 if prob >= 0.5 else 0 for prob in pred_prob])
# record results
model += ['regular feedforward']
fold += [2]
accuracy += [accuracy_score(y_test, pred)]
precision_intox += [precision_recall_fscore_support(y_test, pred)[0][1]]
precision_sober += [precision_recall_fscore_support(y_test, pred)[0][0]]
recall_intox += [precision_recall_fscore_support(y_test, pred)[1][1]]
recall_sober += [precision_recall_fscore_support(y_test, pred)[1][0]]
support_sober += [precision_recall_fscore_support(y_test, pred)[3][0]]
support_intox += [precision_recall_fscore_support(y_test, pred)[3][1]]

Train on 17425 samples, validate on 8712 samples
Epoch 1/50
17425/17425 - 18s - loss: 0.5497 - accuracy: 0.7541 - val_loss: 0.4490 - val_accuracy: 0.7743
Epoch 2/50
17425/17425 - 15s - loss: 0.4811 - accuracy: 0.7685 - val_loss: 0.5688 - val_accuracy: 0.7272
Epoch 3/50
17425/17425 - 15s - loss: 0.4524 - accuracy: 0.7733 - val_loss: 0.5375 - val_accuracy: 0.7126
Epoch 4/50
17425/17425 - 15s - loss: 0.4331 - accuracy: 0.7756 - val_loss: 0.5250 - val_accuracy: 0.7487
Epoch 5/50
17425/17425 - 14s - loss: 0.4194 - accuracy: 0.7840 - val_loss: 0.4979 - val_accuracy: 0.7542
Epoch 6/50
17425/17425 - 14s - loss: 0.4095 - accuracy: 0.7872 - val_loss: 0.4909 - val_accuracy: 0.7495
Epoch 7/50
17425/17425 - 14s - loss: 0.4026 - accuracy: 0.7904 - val_loss: 0.4895 - val_accuracy: 0.7603
Epoch 8/50
17425/17425 - 15s - loss: 0.3900 - accuracy: 0.7956 - val_loss: 0.4924 - val_accuracy: 0.7448
Epoch 9/50
17425/17425 - 15s - loss: 0.3853 - accuracy: 0.8000 - val_loss: 0.4803 - val_accuracy: 0.7444
Epoch 

**Fold 3**

In [21]:
# get data
X_train, X_test, y_train, y_test = data_for_modeling(train_3_indices, test_3_indices)
# fit model
feedforward_3 = train_regular_feedforward(X_train, X_test, y_train, y_test, 'fold_3')
# predict labels
pred_prob = feedforward_3.predict(X_test)
pred = np.array([1 if prob >= 0.5 else 0 for prob in pred_prob])
# record results
model += ['regular feedforward']
fold += [3]
accuracy += [accuracy_score(y_test, pred)]
precision_intox += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[0][1]]
precision_sober += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[0][0]]
recall_intox += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[1][1]]
recall_sober += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[1][0]]
support_sober += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[3][0]]
support_intox += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[3][1]]

Train on 17424 samples, validate on 8713 samples
Epoch 1/50
17424/17424 - 16s - loss: 0.7384 - accuracy: 0.6477 - val_loss: 0.1357 - val_accuracy: 0.9392
Epoch 2/50
17424/17424 - 14s - loss: 0.6767 - accuracy: 0.6666 - val_loss: 0.2138 - val_accuracy: 0.9380
Epoch 3/50
17424/17424 - 14s - loss: 0.6440 - accuracy: 0.6742 - val_loss: 0.1303 - val_accuracy: 0.9564
Epoch 4/50
17424/17424 - 14s - loss: 0.6103 - accuracy: 0.6861 - val_loss: 0.1447 - val_accuracy: 0.9210
Epoch 5/50
17424/17424 - 14s - loss: 0.5858 - accuracy: 0.6930 - val_loss: 0.1172 - val_accuracy: 0.9518
Epoch 6/50
17424/17424 - 14s - loss: 0.5724 - accuracy: 0.6985 - val_loss: 0.1243 - val_accuracy: 0.9492
Epoch 7/50
17424/17424 - 14s - loss: 0.5604 - accuracy: 0.7056 - val_loss: 0.1426 - val_accuracy: 0.9635
Epoch 8/50
17424/17424 - 14s - loss: 0.5493 - accuracy: 0.7084 - val_loss: 0.1520 - val_accuracy: 0.9249
Epoch 9/50
17424/17424 - 13s - loss: 0.5381 - accuracy: 0.7125 - val_loss: 0.1489 - val_accuracy: 0.9543
Epoch 

/Users/yannik/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Model #2 Wide and deep feedforward NN

In [38]:
def train_wide_deep_feedforward(X_train_wide, X_train_deep, X_test_wide, X_test_deep, y_train, y_test, name):
    '''
    Creates wide and deep feedforward NN and fits it to supplied data. 
    Returns fitted model.
    Name specifies filename of model saved at checkpoints.
    '''
    # specify network architecture
    wide_input = keras.layers.Input(shape = (96,), name = 'wide_input')
    deep_input = keras.layers.Input(shape = (3,), name = 'deep_input')
    hidden_1 = keras.layers.Dense(units = 300, activation = "selu", kernel_initializer = "lecun_normal")(deep_input)
    dropout_1 = keras.layers.Dropout(rate = 0.3)(hidden_1)
    hidden_2 = keras.layers.Dense(units = 300, activation = "selu", kernel_initializer = "lecun_normal")(dropout_1)
    dropout_2 = keras.layers.Dropout(rate = 0.3)(hidden_2)
    hidden_3 = keras.layers.Dense(units = 300, activation = "selu", kernel_initializer = "lecun_normal")(dropout_2)
    dropout_3 = keras.layers.Dropout(rate = 0.3)(hidden_3)
    dropout_wide = keras.layers.Dropout(rate = 0.3)(wide_input)
    concat = keras.layers.concatenate([dropout_wide, dropout_3])
    output = keras.layers.Dense(units = 1, activation = "sigmoid")(concat)
    model = keras.Model(inputs = [wide_input, deep_input], outputs = [output])
    
    # compilation
    optimizer = keras.optimizers.SGD(momentum = 0.9, nesterov = True)
    model.compile(loss = 'binary_crossentropy', optimizer = optimizer, metrics = ['accuracy'])
    
    # callbacks
    lr_scheduler = keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=5)
    checkpoint_cb = keras.callbacks.ModelCheckpoint('model_checkpoints/' + 'wide_deep_feedforward_' + name + '.h5', save_best_only = True, save_weights_only = False)
    early_stopping_cb = keras.callbacks.EarlyStopping(patience = 15, restore_best_weights = True)
    
    model.fit((X_train_wide, X_train_deep), y_train, batch_size = 32, epochs = 50, validation_data = ((X_test_wide, X_test_deep), y_test),
                        callbacks = [checkpoint_cb, lr_scheduler, early_stopping_cb], verbose = 2)
    
    return(model)

**Fold 1**

In [49]:
# get data
X_train_wide, X_train_deep, X_test_wide, X_test_deep, y_train, y_test = data_for_modeling(train_1_indices, test_1_indices, multi_input = True)
# fit model
wide_deep_1 = train_wide_deep_feedforward(X_train_wide, X_train_deep, X_test_wide, X_test_deep, y_train, y_test, 'fold_1')
# predict labels
pred_prob = wide_deep_1.predict((X_test_wide, X_test_deep))
pred = np.array([1 if prob >= 0.5 else 0 for prob in pred_prob])
# record results
model += ['wide and deep feedforward']
fold += [1]
accuracy += [accuracy_score(y_test, pred)]
precision_intox += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[0][1]]
precision_sober += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[0][0]]
recall_intox += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[1][1]]
recall_sober += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[1][0]]
support_sober += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[3][0]]
support_intox += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[3][1]]

Train on 17425 samples, validate on 8712 samples
Epoch 1/50
17425/17425 - 17s - loss: 0.3825 - accuracy: 0.8735 - val_loss: 1.3326 - val_accuracy: 0.5680
Epoch 2/50
17425/17425 - 13s - loss: 0.3383 - accuracy: 0.8799 - val_loss: 1.3561 - val_accuracy: 0.5514
Epoch 3/50
17425/17425 - 15s - loss: 0.3159 - accuracy: 0.8813 - val_loss: 1.2525 - val_accuracy: 0.5698
Epoch 4/50
17425/17425 - 14s - loss: 0.2876 - accuracy: 0.8853 - val_loss: 1.1039 - val_accuracy: 0.5899
Epoch 5/50
17425/17425 - 14s - loss: 0.2826 - accuracy: 0.8859 - val_loss: 1.1146 - val_accuracy: 0.5875
Epoch 6/50
17425/17425 - 14s - loss: 0.2847 - accuracy: 0.8856 - val_loss: 1.1267 - val_accuracy: 0.5764
Epoch 7/50
17425/17425 - 15s - loss: 0.2792 - accuracy: 0.8872 - val_loss: 1.1151 - val_accuracy: 0.5658
Epoch 8/50
17425/17425 - 14s - loss: 0.2773 - accuracy: 0.8830 - val_loss: 1.0767 - val_accuracy: 0.5830
Epoch 9/50
17425/17425 - 14s - loss: 0.2794 - accuracy: 0.8865 - val_loss: 1.0530 - val_accuracy: 0.5927
Epoch 

**Fold 2**

In [50]:
# get data
X_train_wide, X_train_deep, X_test_wide, X_test_deep, y_train, y_test = data_for_modeling(train_2_indices, test_2_indices, multi_input = True)
# fit model
wide_deep_2 = train_wide_deep_feedforward(X_train_wide, X_train_deep, X_test_wide, X_test_deep, y_train, y_test, 'fold_2')
# predict labels
pred_prob = wide_deep_2.predict((X_test_wide, X_test_deep))
pred = np.array([1 if prob >= 0.5 else 0 for prob in pred_prob])
# record results
model += ['wide and deep feedforward']
fold += [2]
accuracy += [accuracy_score(y_test, pred)]
precision_intox += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[0][1]]
precision_sober += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[0][0]]
recall_intox += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[1][1]]
recall_sober += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[1][0]]
support_sober += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[3][0]]
support_intox += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[3][1]]

Train on 17425 samples, validate on 8712 samples
Epoch 1/50
17425/17425 - 19s - loss: 0.5239 - accuracy: 0.7757 - val_loss: 0.6090 - val_accuracy: 0.7290
Epoch 2/50
17425/17425 - 14s - loss: 0.4876 - accuracy: 0.7795 - val_loss: 0.5582 - val_accuracy: 0.7452
Epoch 3/50
17425/17425 - 14s - loss: 0.4644 - accuracy: 0.7815 - val_loss: 0.6072 - val_accuracy: 0.7243
Epoch 4/50
17425/17425 - 14s - loss: 0.4683 - accuracy: 0.7774 - val_loss: 0.5963 - val_accuracy: 0.7207
Epoch 5/50
17425/17425 - 14s - loss: 0.4598 - accuracy: 0.7804 - val_loss: 0.5405 - val_accuracy: 0.7239
Epoch 6/50
17425/17425 - 14s - loss: 0.4476 - accuracy: 0.7798 - val_loss: 0.5150 - val_accuracy: 0.7332
Epoch 7/50
17425/17425 - 16s - loss: 0.4492 - accuracy: 0.7806 - val_loss: 0.5272 - val_accuracy: 0.7327
Epoch 8/50
17425/17425 - 14s - loss: 0.4492 - accuracy: 0.7818 - val_loss: 0.5446 - val_accuracy: 0.7338
Epoch 9/50
17425/17425 - 15s - loss: 0.4443 - accuracy: 0.7814 - val_loss: 0.5150 - val_accuracy: 0.7415
Epoch 

**Fold 3**

In [51]:
# get data
X_train_wide, X_train_deep, X_test_wide, X_test_deep, y_train, y_test = data_for_modeling(train_3_indices, test_3_indices, multi_input = True)
# fit model
wide_deep_3 = train_wide_deep_feedforward(X_train_wide, X_train_deep, X_test_wide, X_test_deep, y_train, y_test, 'fold_3')
# predict labels
pred_prob = wide_deep_3.predict((X_test_wide, X_test_deep))
pred = np.array([1 if prob >= 0.5 else 0 for prob in pred_prob])
# record results
model += ['wide and deep feedforward']
fold += [3]
accuracy += [accuracy_score(y_test, pred)]
precision_intox += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[0][1]]
precision_sober += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[0][0]]
recall_intox += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[1][1]]
recall_sober += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[1][0]]
support_sober += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[3][0]]
support_intox += [precision_recall_fscore_support(y_test, pred, zero_division = 1)[3][1]]

Train on 17424 samples, validate on 8713 samples
Epoch 1/50
17424/17424 - 16s - loss: 0.7029 - accuracy: 0.6709 - val_loss: 0.2942 - val_accuracy: 0.9387
Epoch 2/50
17424/17424 - 13s - loss: 0.6333 - accuracy: 0.6820 - val_loss: 0.2751 - val_accuracy: 0.9475
Epoch 3/50
17424/17424 - 14s - loss: 0.6208 - accuracy: 0.6769 - val_loss: 0.1826 - val_accuracy: 0.9558
Epoch 4/50
17424/17424 - 16s - loss: 0.6096 - accuracy: 0.6841 - val_loss: 0.1915 - val_accuracy: 0.9661
Epoch 5/50
17424/17424 - 14s - loss: 0.6080 - accuracy: 0.6841 - val_loss: 0.2098 - val_accuracy: 0.9604
Epoch 6/50
17424/17424 - 15s - loss: 0.6073 - accuracy: 0.6810 - val_loss: 0.2625 - val_accuracy: 0.9539
Epoch 7/50
17424/17424 - 14s - loss: 0.5996 - accuracy: 0.6838 - val_loss: 0.1790 - val_accuracy: 0.9571
Epoch 8/50
17424/17424 - 13s - loss: 0.6022 - accuracy: 0.6861 - val_loss: 0.1928 - val_accuracy: 0.9570
Epoch 9/50
17424/17424 - 13s - loss: 0.6091 - accuracy: 0.6835 - val_loss: 0.1924 - val_accuracy: 0.9601
Epoch 

## Model #3 Multi-task feedforward NN

In [57]:
def train_multi_task_feedforward(X_train, X_test, y_train_classification, y_train_regression, y_test_classification, y_test_regression, name):
    '''
    Creates multi-task feedforward NN and fits it to supplied data. 
    Returns fitted model.
    Name specifies filename of model saved at checkpoints.
    The two tasks are classification of TAC and regression of TAC.
    '''
    # specify network architecture
    input_layer = keras.layers.Input(shape = (99,), name = 'input_layer')
    hidden_1 = keras.layers.Dense(units = 300, activation = "selu", kernel_initializer = "lecun_normal")(input_layer)
    dropout_1 = keras.layers.Dropout(rate = 0.3)(hidden_1)
    hidden_2 = keras.layers.Dense(units = 300, activation = "selu", kernel_initializer = "lecun_normal")(dropout_1)
    dropout_2 = keras.layers.Dropout(rate = 0.3)(hidden_2)
    hidden_3 = keras.layers.Dense(units = 300, activation = "selu", kernel_initializer = "lecun_normal")(dropout_2)
    dropout_3 = keras.layers.Dropout(rate = 0.3)(hidden_3)
    classifier_output = keras.layers.Dense(units = 1, activation = "sigmoid", name = 'classifier')(dropout_3)
    regression_output = keras.layers.Dense(units = 1, activation = "sigmoid", name = 'regressor')(dropout_3)
    model = keras.Model(inputs = [input_layer], outputs = [classifier_output, regression_output])
    
    # compilation
    optimizer = keras.optimizers.SGD(momentum = 0.9, nesterov = True)
    model.compile(loss = ['binary_crossentropy', 'mean_squared_error'], optimizer = optimizer, 
                  metrics = [['accuracy'], ['mse']])
    
    # callbacks
    lr_scheduler = keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=5)
    checkpoint_cb = keras.callbacks.ModelCheckpoint('model_checkpoints/' + 'multi_task_feedforward_' + name + '.h5', save_best_only = True, save_weights_only = False)
    early_stopping_cb = keras.callbacks.EarlyStopping(patience = 15, restore_best_weights = True)
    
    model.fit(X_train, [y_train_classification, y_train_regression], batch_size = 32, epochs = 50, 
              validation_data = (X_test, [y_test_classification, y_test_regression]), 
              callbacks = [lr_scheduler, early_stopping_cb, checkpoint_cb], verbose = 2)
    
    return(model)

**Fold 1**

In [70]:
# get data
X_train, X_test, y_train_classification, y_train_regression, y_test_classification, y_test_regression = data_for_modeling(train_1_indices, test_1_indices, multi_output = True)
# fit model
multi_task_1 = train_multi_task_feedforward(X_train, X_test, y_train_classification, y_train_regression, y_test_classification, y_test_regression, 'fold_1')
# get predictions
pred_prob, _ = multi_task_1.predict(X_test)
pred = np.array([1 if prob >= 0.5 else 0 for prob in pred_prob])
# record results
model += ['multi-task feedforward']
fold += [1]
accuracy += [accuracy_score(y_test_classification, pred)]
precision_intox += [precision_recall_fscore_support(y_test_classification, pred, zero_division = 1)[0][1]]
precision_sober += [precision_recall_fscore_support(y_test_classification, pred, zero_division = 1)[0][0]]
recall_intox += [precision_recall_fscore_support(y_test_classification, pred, zero_division = 1)[1][1]]
recall_sober += [precision_recall_fscore_support(y_test_classification, pred, zero_division = 1)[1][0]]
support_sober += [precision_recall_fscore_support(y_test_classification, pred, zero_division = 1)[3][0]]
support_intox += [precision_recall_fscore_support(y_test_classification, pred, zero_division = 1)[3][1]]

Train on 17425 samples, validate on 8712 samples
Epoch 1/50
17425/17425 - 20s - loss: 0.3227 - classifier_loss: 0.3076 - regressor_loss: 0.0150 - classifier_accuracy: 0.8767 - regressor_mse: 0.0150 - val_loss: 1.7067 - val_classifier_loss: 1.6972 - val_regressor_loss: 0.0075 - val_classifier_accuracy: 0.5865 - val_regressor_mse: 0.0075
Epoch 2/50
17425/17425 - 16s - loss: 0.2491 - classifier_loss: 0.2447 - regressor_loss: 0.0043 - classifier_accuracy: 0.8969 - regressor_mse: 0.0043 - val_loss: 1.4237 - val_classifier_loss: 1.4152 - val_regressor_loss: 0.0073 - val_classifier_accuracy: 0.5948 - val_regressor_mse: 0.0073
Epoch 3/50
17425/17425 - 15s - loss: 0.2207 - classifier_loss: 0.2168 - regressor_loss: 0.0038 - classifier_accuracy: 0.9072 - regressor_mse: 0.0038 - val_loss: 1.5511 - val_classifier_loss: 1.5430 - val_regressor_loss: 0.0076 - val_classifier_accuracy: 0.5704 - val_regressor_mse: 0.0076
Epoch 4/50
17425/17425 - 15s - loss: 0.2087 - classifier_loss: 0.2050 - regressor_lo

**Fold 2**

In [71]:
# get data
X_train, X_test, y_train_classification, y_train_regression, y_test_classification, y_test_regression = data_for_modeling(train_2_indices, test_2_indices, multi_output = True)
# fit model
multi_task_2 = train_multi_task_feedforward(X_train, X_test, y_train_classification, y_train_regression, y_test_classification, y_test_regression, 'fold_2')
# get predictions
pred_prob, _ = multi_task_2.predict(X_test)
pred = np.array([1 if prob >= 0.5 else 0 for prob in pred_prob])
# record results
model += ['multi-task feedforward']
fold += [2]
accuracy += [accuracy_score(y_test_classification, pred)]
precision_intox += [precision_recall_fscore_support(y_test_classification, pred, zero_division = 1)[0][1]]
precision_sober += [precision_recall_fscore_support(y_test_classification, pred, zero_division = 1)[0][0]]
recall_intox += [precision_recall_fscore_support(y_test_classification, pred, zero_division = 1)[1][1]]
recall_sober += [precision_recall_fscore_support(y_test_classification, pred, zero_division = 1)[1][0]]
support_sober += [precision_recall_fscore_support(y_test_classification, pred, zero_division = 1)[3][0]]
support_intox += [precision_recall_fscore_support(y_test_classification, pred, zero_division = 1)[3][1]]

Train on 17425 samples, validate on 8712 samples
Epoch 1/50
17425/17425 - 18s - loss: 0.5186 - classifier_loss: 0.5015 - regressor_loss: 0.0170 - classifier_accuracy: 0.7696 - regressor_mse: 0.0170 - val_loss: 0.5953 - val_classifier_loss: 0.5877 - val_regressor_loss: 0.0061 - val_classifier_accuracy: 0.7127 - val_regressor_mse: 0.0061
Epoch 2/50
17425/17425 - 15s - loss: 0.4556 - classifier_loss: 0.4505 - regressor_loss: 0.0053 - classifier_accuracy: 0.7857 - regressor_mse: 0.0053 - val_loss: 0.5460 - val_classifier_loss: 0.5384 - val_regressor_loss: 0.0062 - val_classifier_accuracy: 0.7423 - val_regressor_mse: 0.0062
Epoch 3/50
17425/17425 - 14s - loss: 0.4365 - classifier_loss: 0.4318 - regressor_loss: 0.0048 - classifier_accuracy: 0.7942 - regressor_mse: 0.0048 - val_loss: 0.5614 - val_classifier_loss: 0.5539 - val_regressor_loss: 0.0059 - val_classifier_accuracy: 0.7599 - val_regressor_mse: 0.0059
Epoch 4/50
17425/17425 - 14s - loss: 0.4252 - classifier_loss: 0.4206 - regressor_lo

**Fold 3**

In [72]:
# get data
X_train, X_test, y_train_classification, y_train_regression, y_test_classification, y_test_regression = data_for_modeling(train_3_indices, test_3_indices, multi_output = True)
# fit model
multi_task_3 = train_multi_task_feedforward(X_train, X_test, y_train_classification, y_train_regression, y_test_classification, y_test_regression, 'fold_3')
# get predictions
pred_prob, _ = multi_task_3.predict(X_test)
pred = np.array([1 if prob >= 0.5 else 0 for prob in pred_prob])
# record results
model += ['multi-task feedforward']
fold += [3]
accuracy += [accuracy_score(y_test_classification, pred)]
precision_intox += [precision_recall_fscore_support(y_test_classification, pred, zero_division = 1)[0][1]]
precision_sober += [precision_recall_fscore_support(y_test_classification, pred, zero_division = 1)[0][0]]
recall_intox += [precision_recall_fscore_support(y_test_classification, pred, zero_division = 1)[1][1]]
recall_sober += [precision_recall_fscore_support(y_test_classification, pred, zero_division = 1)[1][0]]
support_sober += [precision_recall_fscore_support(y_test_classification, pred, zero_division = 1)[3][0]]
support_intox += [precision_recall_fscore_support(y_test_classification, pred, zero_division = 1)[3][1]]

Train on 17424 samples, validate on 8713 samples
Epoch 1/50
17424/17424 - 17s - loss: 0.7052 - classifier_loss: 0.6869 - regressor_loss: 0.0189 - classifier_accuracy: 0.6733 - regressor_mse: 0.0189 - val_loss: 0.1390 - val_classifier_loss: 0.1407 - val_regressor_loss: 4.1545e-04 - val_classifier_accuracy: 0.9401 - val_regressor_mse: 4.1220e-04
Epoch 2/50
17424/17424 - 14s - loss: 0.6444 - classifier_loss: 0.6366 - regressor_loss: 0.0078 - classifier_accuracy: 0.6917 - regressor_mse: 0.0078 - val_loss: 0.2659 - val_classifier_loss: 0.2687 - val_regressor_loss: 5.4181e-04 - val_classifier_accuracy: 0.8989 - val_regressor_mse: 5.4079e-04
Epoch 3/50
17424/17424 - 14s - loss: 0.6268 - classifier_loss: 0.6200 - regressor_loss: 0.0068 - classifier_accuracy: 0.7058 - regressor_mse: 0.0068 - val_loss: 0.1573 - val_classifier_loss: 0.1615 - val_regressor_loss: 7.7563e-04 - val_classifier_accuracy: 0.9374 - val_regressor_mse: 7.6273e-04
Epoch 4/50
17424/17424 - 14s - loss: 0.5976 - classifier_los

# Results

In [73]:
pd.DataFrame(results)

,model,fold,accuracy,precision (intoxicated),precision (sober),recall (intoxicated),recall (sober),support (sober),support (intox)
0,regular feedforward,1,0.565771,0.540795,0.575482,0.331241,0.763214,4730,3982
1,regular feedforward,2,0.774334,0.584813,0.857049,0.640994,0.825472,6297,2415
2,regular feedforward,3,0.951796,0.000000,1.000000,1.000000,0.951796,8713,0
3,wide and deep feedforward,1,0.580234,0.564974,0.586379,0.354847,0.769979,4730,3982
4,wide and deep feedforward,2,0.739096,0.546834,0.779600,0.343271,0.890900,6297,2415
5,wide and deep feedforward,3,0.954092,0.000000,1.000000,1.000000,0.954092,8713,0
6,multi-task feedforward,1,0.594812,0.577610,0.603448,0.422401,0.739958,4730,3982
7,multi-task feedforward,2,0.747704,0.541114,0.837477,0.591304,0.807686,6297,2415
8,multi-task feedforward,3,0.958109,0.000000,1.000000,1.000000,0.958109,8713,0


In [76]:
pd.DataFrame(results).groupby('model', as_index = False).agg(np.mean).drop(['fold', 'support (sober)', 'support (intox)'], axis = 1)

,model,accuracy,precision (intoxicated),precision (sober),recall (intoxicated),recall (sober)
0,multi-task feedforward,0.766875,0.372908,0.813642,0.671235,0.835251
1,regular feedforward,0.763967,0.375203,0.810844,0.657411,0.846827
2,wide and deep feedforward,0.757807,0.370603,0.788660,0.566039,0.871657
